In [ ]:
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from keras.models import Sequential
from keras.layers import Dense
import keras.activations,keras.optimizers,keras.losses

In [ ]:
data=pd.read_csv("/kaggle/input/iot-dataset-for-intrusion-detection-systems-ids/BoTNeTIoT-L01-v2.csv")
print(data.columns)
print(data.describe())
print(data.isna().sum())
print(data.info())
print(len(data.columns))


In [ ]:
for i in data.select_dtypes(include='number').columns.values:
    sn.boxplot(data[i])
    plt.show()

In [ ]:
for i in data.select_dtypes(include='object').columns.values:
    if len(data[i].value_counts()) <=10:
        val=data[i].value_counts().values
        index=data[i].value_counts().index
        plt.pie(val,labels=index,autopct='%1.1f%%')
        plt.title(f'The PIE Chart information of {i} column')
        plt.show()

In [ ]:
for i in data.select_dtypes(include='object').columns.values:
    print(data[i].value_counts())
    print("--------------------------------")


In [ ]:
lab=LabelEncoder()
for i in data.select_dtypes(include='object').columns.values:
    data[i]=lab.fit_transform(data[i])


In [ ]:
x={}
X=[]
for i in data.columns.values:
    data['z-scores']=(data[i]-data[i].mean())/(data[i].std())
    outliers=np.abs(data['z-scores'] > 3).sum()
    x[i]=outliers


In [ ]:
for keys,values in x.items():
    if values>0:
        X.append(keys)
print(x)

In [ ]:
x=[]

thresh=2
for i in data[X].columns.values:
    upper=data[i].mean()+thresh*data[i].std()
    lower=data[i].mean()-thresh*data[i].std()
    data=data[(data[i]>lower)&(data[i]<upper)]

print(len(data))

In [ ]:
corr=data.corr()['Attack']
corr=corr.drop(['Attack','z-scores','label'])
for i in corr.index:
    print(i)
    if corr[i] > 0.25:
        x.append(i)

In [ ]:
X=data[x]
data['Attack']=lab.fit_transform(data['Attack'])
Y=data['Attack']
x_train,x_test,y_train,y_test=train_test_split(X,Y)

lr = LogisticRegression(max_iter=500)
lr.fit(x_train, y_train)
print('The logistic regression: ', lr.score(x_test, y_test))

lgb = LGBMClassifier() 
lgb.fit(x_train, y_train)
print('The LGB', lgb.score(x_test, y_test))

tree = DecisionTreeClassifier(criterion='entropy', max_depth=5)
tree.fit(x_train, y_train)
print('Dtree ', tree.score(x_test,y_test))

In [ ]:
x=data[x]
print(len(x))
y=pd.get_dummies(data['Attack'])
x_trin,x_tst,y_trin,y_tst=train_test_split(x,y)
models=Sequential()
models.add(Dense(units=x.shape[1],input_dim=x.shape[1],activation=keras.activations.relu))
models.add(Dense(units=x.shape[1],activation=keras.activations.softmax))
models.add(Dense(units=x.shape[1],activation=keras.activations.softmax))
models.add(Dense(units=x.shape[1],activation=keras.activations.softmax))
models.add(Dense(units=y.shape[1],activation=keras.activations.softmax))
models.compile(optimizer='adam',loss=keras.losses.categorical_crossentropy,metrics='accuracy')
hist=models.fit(x_trin,y_trin,batch_size=500,epochs=50)
plt.plot(hist.history['accuracy'], label='training accuracy', marker='o', color='red')
plt.plot(hist.history['loss'], label='loss', marker='o', color='darkblue')
plt.title('Training Vs  Validation accuracy with adam optimizer')
plt.legend()
plt.show()

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, LSTM, GRU, Dense
from sklearn.metrics import accuracy_score, f1_score
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split

# Define your CNN model
def create_cnn_model(input_shape, num_classes):
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    return model

# Define your LSTM model
def create_lstm_model(input_shape, num_classes):
    model = Sequential()
    model.add(LSTM(64, input_shape=input_shape))
    model.add(Dense(num_classes, activation='softmax'))
    return model

# Define your GRU model
def create_gru_model(input_shape, num_classes):
    model = Sequential()
    model.add(GRU(64, input_shape=input_shape))
    model.add(Dense(num_classes, activation='softmax'))
    return model

# Load and preprocess  IoT intrusion dataset
# Replace this with your dataset loading and preprocessing code
X, y = load_and_preprocess_data()

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create CNN, LSTM, and GRU models
input_shape = X_train.shape[1:]
num_classes = len(np.unique(y_train))

cnn_model = create_cnn_model(input_shape, num_classes)
lstm_model = create_lstm_model(input_shape, num_classes)
gru_model = create_gru_model(input_shape, num_classes)

# Compile models
cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
gru_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train models
cnn_model.fit(X_train, y_train, epochs=10, batch_size=64)
lstm_model.fit(X_train, y_train, epochs=10, batch_size=64)
gru_model.fit(X_train, y_train, epochs=10, batch_size=64)

# Make predictions
cnn_pred = cnn_model.predict(X_test)
lstm_pred = lstm_model.predict(X_test)
gru_pred = gru_model.predict(X_test)

# Create an ensemble model with both hard and soft voting
ensemble_hard = VotingClassifier(estimators=[
    ('cnn', cnn_model),
    ('lstm', lstm_model),
    ('gru', gru_model)
], voting='hard')

ensemble_soft = VotingClassifier(estimators=[
    ('cnn', cnn_model),
    ('lstm', lstm_model),
    ('gru', gru_model)
], voting='soft')

ensemble_hard.fit(X_train, y_train)
ensemble_soft.fit(X_train, y_train)

ensemble_hard_pred = ensemble_hard.predict(X_test)
ensemble_soft_pred = ensemble_soft.predict(X_test)

# Evaluate the models
cnn_accuracy = accuracy_score(np.argmax(y_test, axis=1), np.argmax(cnn_pred, axis=1))
lstm_accuracy = accuracy_score(np.argmax(y_test, axis=1), np.argmax(lstm_pred, axis=1))
gru_accuracy = accuracy_score(np.argmax(y_test, axis=1), np.argmax(gru_pred, axis_1))
ensemble_hard_accuracy = accuracy_score(y_test, ensemble_hard_pred)
ensemble_soft_accuracy = accuracy_score(y_test, ensemble_soft_pred)

cnn_f1 = f1_score(np.argmax(y_test, axis=1), np.argmax(cnn_pred, axis=1), average='weighted')
lstm_f1 = f1_score(np.argmax(y_test, axis=1), np.argmax(lstm_pred, axis=1), average='weighted')
gru_f1 = f1_score(np.argmax(y_test, axis=1), np.argmax(gru_pred, axis=1), average='weighted')
ensemble_hard_f1 = f1_score(y_test, ensemble_hard_pred, average='weighted')
ensemble_soft_f1 = f1_score(y_test, ensemble_soft_pred, average='weighted')

# Print the results
print(f"CNN Accuracy: {cnn_accuracy}")
print(f"LSTM Accuracy: {lstm_accuracy}")
print(f"GRU Accuracy: {gru_accuracy}")
print(f"Ensemble (Hard Voting) Accuracy: {ensemble_hard_accuracy}")
print(f"Ensemble (Soft Voting) Accuracy: {ensemble_soft_accuracy}")

print(f"CNN F1-Score: {cnn_f1}")
print(f"LSTM F1-Score: {lstm_f1}")
print(f"GRU F1-Score: {gru_f1}")
print(f"Ensemble (Hard Voting) F1-Score: {ensemble_hard_f1}")
print(f"Ensemble (Soft Voting) F1-Score: {ensemble_soft_f1}")
